# Asset management assignment

**Question 1** : 
For this exercise use the file ”Tickers”, which specifies the tickers of the 50 stocks you should use
given your group number. Retrieve monthly Book-to-Market data and monthly prices for your
stocks from 1980. Present annualized performance statistics (average return, volatility, Sharpe
ratio, skewness, kurtosis and max drawdown) for your stocks

In [182]:
import pandas as pd
import numpy as np
import matplotlib as plt
import scipy.stats
import yfinance as yf

In [183]:
data_set_bme = pd.read_csv("data-bme.csv")
data_set_prc = pd.read_csv("data-prc.csv")

In [192]:
data_set_bme = data_set_bme.rename(columns={'public_date': 'date', 'TICKER': 'ticker'}) #we will assume the Public date as the date of our signal
data_set_prc = data_set_prc.rename(columns={'PERMNO': 'permno'}) 

In [185]:
# Define a list of tickers and a start and end date for the price data
tickers = ['AIZ', 'ARE', 'ATO', 'CARR', 'CBOE', 'CE', 'CMCSA', 'CTVA', 'DG', 'DPZ', 'ECL', 'EIX', 'EQIX', 'EVRG', 'FCX', 'FFIV', 'FIS', 'FLT', 'GLW', 'GRMN', 'HAL', 'HAS', 'HII', 'HUM', 'HWM', 'INVH', 'IT', 'ITW', 'JBHT', 'K', 'KEY', 'LEN', 'LIN', 'LMT', 'MCK', 'MCO', 'MHK', 'MOH', 'NSC', 'ODFL', 'PEG', 'PGR', 'QCOM', 'SPGI', 'STT', 'TROW', 'TSN', 'UDR', 'V', 'WY']
start_date = '1980-01-01'
end_date = '2022-12-31'

# Use yfinance to fetch the stock prices for the tickers and date range
prices_df = yf.download(tickers, start=start_date, end=end_date, group_by='ticker')

# Reset the index to create 'Date' and 'TICKER' columns
prices_df = prices_df.reset_index()

# Rename the columns to match the column names in the original dataframe
prices_df = prices_df.rename(columns={'Close': 'Price', 'Ticker': 'TICKER'})

[*********************100%***********************]  50 of 50 completed


In [187]:
cols = ['Open', 'High', 'Low', 'Adj Close', 'Volume']

prices_df = prices_df.drop(columns=cols,level=1)

In [188]:
prices_df

,Date,V,AIZ,CE,FFIV,ODFL,QCOM,STT,LMT,SPGI,...,GRMN,PGR,MCK,EVRG,GLW,ITW,HAL,EIX,TROW,PEG
,,Price,Price,Price,Price,Price,Price,Price,Price,Price,...,Price,Price,Price,Price,Price,Price,Price,Price,Price,Price
0,1980-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.399796,1.695313,...,NaN,NaN,NaN,8.8125,NaN,0.789063,9.902370,6.062500,NaN,6.375000
1,1980-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.617076,1.687500,...,NaN,NaN,NaN,8.7500,NaN,0.796875,9.798134,6.031250,NaN,6.416667
2,1980-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.885481,1.695313,...,NaN,NaN,NaN,8.3750,NaN,0.796875,9.976823,6.125000,NaN,6.458333
3,1980-01-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.102761,1.679688,...,NaN,NaN,NaN,8.5000,NaN,0.796875,9.902370,6.093750,NaN,6.458333
4,1980-01-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.473415,1.695313,...,NaN,NaN,NaN,8.5000,NaN,0.812500,10.244858,6.062500,NaN,6.416667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10838,2022-12-23,205.830002,125.029999,102.559998,141.009995,286.100006,110.839996,77.489998,483.290009,332.940002,...,91.260002,129.440002,383.670013,63.3400,31.750000,221.970001,39.090000,65.180000,111.120003,61.080002
10839,2022-12-27,206.289993,124.570000,103.750000,141.190002,286.769989,109.459999,77.150002,485.500000,331.320007,...,91.779999,130.169998,380.190002,63.4800,31.900000,223.740005,39.270000,65.820000,110.730003,61.810001
10840,2022-12-28,204.990005,123.959999,100.500000,138.839996,282.250000,106.970001,76.839996,483.220001,329.600006,...,90.250000,129.720001,377.510010,63.2300,31.420000,220.429993,38.080002,65.019997,108.080002,61.290001


In [193]:
cols = ['permno', 'adate', 'qdate']
data_set_bme.drop(columns=cols)

,date,bm,ticker
0,1986-06-30,0.110,NaN
1,1986-07-31,0.121,NaN
2,1986-08-31,0.137,TROW
3,1986-09-30,0.137,TROW
4,1986-10-31,0.137,TROW
...,...,...,...
19856,2022-08-31,0.293,CBOE
19857,2022-09-30,0.293,CBOE
19858,2022-10-31,0.293,CBOE
19859,2022-11-30,0.284,CBOE


In [ ]:
merged_df = pd.merge(data_set_bme, data_set_prc, on=['date', 'permno', 'TICKER'], how='left') 
new_df = merged_df[['permno','TICKER','date', 'PRC','bm']] 
df = new_df.rename(columns={'TICKER': 'Ticker', 'permno': 'Permno', 'date': 'Date', 'PRC': 'Price', 'bm': 'Bme'}) 
df.set_index('Permno', inplace=True) 
df =df.dropna() 
df

In [ ]:
my_list = ['AIZ', 'ARE', 'ATO', 'CARR', 'CBOE', 'CE', 'CMCSA', 'CTVA', 'DG', 'DPZ', 'ECL', 'EIX', 'EQIX', 'EVRG', 'FCX', 'FFIV', 'FIS', 'FLT', 'GLW', 'GRMN', 'HAL', 'HAS', 'HII', 'HUM', 'HWM', 'INVH', 'IT', 'ITW', 'JBHT', 'K', 'KEY', 'LEN', 'LIN', 'LMT', 'MCK', 'MCO', 'MHK', 'MOH', 'NSC', 'ODFL', 'PEG', 'PGR', 'QCOM', 'SPGI', 'STT', 'TROW', 'TSN', 'UDR', 'V', 'WY']
len(my_list) #check the Ticker assigned to our group

In [ ]:
#match our list of Ticker with the data retrieve because the latter contains more tickers for some reason
df = df[df['Ticker'].isin(my_list)]
unique_ticker = df['Ticker'].unique()
print(len(unique_ticker))
print(df['Ticker'].unique())

In [ ]:
#Find the ticker which aren't in the list
for x in my_list:
    if x not in unique_ticker:
        print(x)

#After doing some research, we found that some of these tickers (INVH and UDR) aren't available on WRDS and that the others have been dropped with .dropna()

In [ ]:
df.describe() #We see that some prices are negative, the negative prices indicate a missing data for the closing price 

In [ ]:
#Check the rows with negative prices DECIDE HOW TO DEAL WITH IT
negative = df[df['Price'] < 0]
negative

In [ ]:
#Put prices in absolute values, we will assume the average bid/ask price (presented as a negative value) as the closing price
df['Price'].abs()

In [ ]:
def calc_annualized_stats(returns):
    avg_return = np.mean(returns)
    volatility = np.std(returns)
    ann_volatility = np.sqrt(12) * volatility
    sharpe_ratio = np.divide(avg_return, ann_volatility)
    skewness = scipy.stats.skew(returns)
    kurtosis = scipy.stats.kurtosis(returns, fisher=False)

    peak = 1
    trough = 1
    max_dd = 0
    for ret in returns:
        peak = max(peak, 1 + ret)
        dd = (trough - peak) / peak
        max_dd = min(max_dd, dd)
        trough = min(trough, 1 + ret)

    ann_max_dd = -np.power(1 + max_dd, 12) + 1

    stats = {'Average Return': avg_return,
             'Volatility': volatility,
             'Annualized Volatility': ann_volatility,
             'Sharpe Ratio': sharpe_ratio,
             'Skewness': skewness,
             'Kurtosis': kurtosis,
             'Max Drawdown': max_dd,
             'Annualized Max Drawdown': ann_max_dd}
    
    return stats

# Calculate annualized performance statistics for each stock
results = []
for ticker in df['Ticker'].unique():
    stock_df = df[df['Ticker'] == ticker]    
    returns = stock_df['Price'].pct_change()    
    returns = returns.iloc[1:]    
    stats = calc_annualized_stats(returns)
    stats['Ticker'] = ticker
    results.append(stats)
    
results_df = pd.DataFrame(results)
results_df.set_index('Ticker', inplace=True)
results_df.head()


**Question 2** : For each stock and each date, build a Momentum indicator which is defined as the cumulated
return over the past six months (note that, by construction, during the first six months of
the sample it will not be possible to build such momentum indicator). Use the characteristics
Book-to-Market and Momentum to build tercile portfolios, called: Value high, Value med and
Value low for Value (book-to-market); and MoM high, MoM med and MoM low for Momentum. To construct the six portfolios, at each date and for each one of the two characteristic, split
your 50 stocks in three equally weighted groups of similar sizes (example: for momentum in Jan
2000 you may have 16 stocks in the low Momentum group, 15 in the Middle group, 15 in the
high Momentum group and 4 stocks with missing observations at this date). Present annualized
performance statistics for the 6 portfolios.


In [ ]:
def get_rets():
    for ticker in df['Ticker'].unique():
        stock_df = df[df['Ticker'] == ticker]    
        returns = stock_df['Price'].pct_change()
    return returns
rets = get_rets()

In [ ]:
df_grouped = df.groupby('Ticker', group_keys=True).apply(lambda x: x.sort_values('Date'))
df_grouped['Returns'] = df_grouped['Price'].pct_change()
df_rets = df_grouped.dropna()
df_rets = df_rets.drop(columns='Ticker')

def momentum_6m(stock_df):
    cum_returns = stock_df['Returns'].add(1).rolling(window=6).apply(np.prod, raw = True) - 1 
    return cum_returns

df_rets['Mom'] = momentum_6m(df_rets)
df_cum = df_rets.dropna()
df_cum = df_cum.reset_index(level=1)
df_cum = df_cum.drop(columns='Permno')

In [ ]:
df_cum['Returns'].describe()

In [ ]:
df_cum['Mom'].describe()

In [ ]:
def tercile_port(df,char_col):
    df[char_col+'_tercile'] = pd.qcut(df[char_col], 3, labels=['low', 'med', 'high'])
    grouped = df.groupby(['Date', char_col+'_tercile'])
    portfolio_returns = grouped.apply(lambda x:np.mean(x['Mom']))
    return portfolio_returns

tercile_port(df_cum,"Mom")
tercile_port(df_cum,"Bme")
df_cum

In [ ]:
groups_mom = df_cum.groupby("Mom_tercile")
groups_bme = df_cum.groupby("Bme_tercile")

low_df_bme = groups_bme.get_group("low")
med_df_bme = groups_bme.get_group("med")
high_df_bme = groups_bme.get_group("high")

low_df_mom = groups_mom.get_group("low")
med_df_mom = groups_mom.get_group("med")
high_df_mom = groups_mom.get_group("high")


In [ ]:
def set_date_as_index(dataf):
    dataf.set_index('Date', inplace=True)

list_of_ports = [low_df_bme,med_df_bme,high_df_bme,low_df_mom,med_df_mom,high_df_mom]

for x in list_of_ports:
    set_date_as_index(x)

**Question 3** : Build two long-short portfolios for Value and Momentum using the tercile portfolios. Backtest
the portfolio and an equal weighed portfolio investing in the 50 stocks. Plot cumulated returns
and present annualized performance statistics

In [ ]:
grouped = df_cum.groupby('Date')

max_bme = grouped['Bme'].max()
min_bme = grouped['Bme'].min()

max_mom = grouped['Mom'].max()
min_mom = grouped['Mom'].min()

bme_port = pd.DataFrame({'Hi_Bme': max_bme, 'Lo_Bme': min_bme})
mom_port = pd.DataFrame({'Hi_Bme': max_mom, 'Lo_Bme': min_mom})

In [ ]:
bme_port

In [ ]:
def backtest_ws(rets, weighting, estimation_window=23, **args):
    """
    Backtests a given weighting scheme, given some parameters:
    rets : asset returns to use to build the portfolio
    estimation_window: the window to use to estimate parameters
    weighting: the weighting scheme to use, must be a function that takes "rets", and a variable number of arguments
    """
    n_periods = rets.shape[0]
    # list of overlapping rolling return windows for estimation
    windows = [(start, start+estimation_window) for start in range(n_periods-estimation_window+1)]
    # list of portfolio weights over time
    weights = [weighting(rets.iloc[win[0]:win[1]], **args) for win in windows]
    # convert to DataFrame
    weights = pd.DataFrame(weights, index=rets.iloc[estimation_window-1:].index, columns=rets.columns)
    returns = (weights * rets).sum(axis="columns",  min_count=1) #mincount is to generate NAs if all inputs are NAs
    return returns

def weight_ew(r):
    w = [ 1/len(r.columns) for _ in range(len(r.columns))]
    w = np.array(w)
    return w

In [ ]:
weight_ew(mom_port)

In [ ]:
backtest_mom = backtest_ws(mom_port, weighting=weight_ew,estimation_window=12)
backtest_bme = backtest_ws(bme_port, weighting=weight_ew,estimation_window=12)

In [ ]:
backtest_bme

In [ ]:
btr = pd.DataFrame({"Bme": backtest_bme,"Mom": backtest_mom})
(1+btr).cumprod().plot(figsize=(12,5))

**Question 4** :  Naturally, neither Value nor Momentum is the perfect predictor of returns. To combine the
information that each characteristic contains, construct a composite characteristics. To this end,
standardize in each month each characteristic to have a cross-sectional mean (standard deviation)
of zero (one) and combine them into a single characteristic Sit defined as:

\begin{equation}
S_{it} = \frac{BM_{it} - E_t[BM_{it}]}{\sigma_t (BM_{it})} + \frac{Mom_{it} - E_t[Mom_{it}]}{\sigma_t (Mom_{it})}
\end{equation}

where the Mean and Standard deviation are taken over all stocks in the cross-section of month
t. Now, sort the stocks on the composite characteristic and construct the long-short portfolio
using tercile portfolios on this charateristics .Backtest this long-short portfolio. Plot cumulated
returns and present annualized performance statistics for this long-short portfolio.

**Question 5** : Discuss how the performance of the strategy at the previous point compares to a mixed strategy
that simply invests 50% in the the value long-short portfolio and %50 in the momentum longshort portfolio

**Question 6** : Consider the long-short portfolios on Value, Momentum, the mixed charateristic S ans the equally
weighted portfolio. Run performance analysis of this strategies against FF5M model. Comment
on the alphas, the factor exposures and the R2 of the factor regressions.

**Question 7** : Is the performance analysis (α) driven mostly by one of the two legs (long ot short)? Discuss
the implications for trading on long-only versus long-short tercile portfolios.

**Question 8** : Discuss theoretically how to modify the construction of the long-short Value portfolio in order
to make it market neutral (i.e. uncorrelated with returns on the market portfolio) over a rolling
widow of 3 years.